In [1]:
print("Hello LangChain!!")

Hello LangChain!!


In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[50:])


k7JQaf


In [4]:
# prompt + llm + output 

# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [5]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [8]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    # model="openai/gpt-oss-120b",
    temperature=0.7
)
print(type(llm))
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x000002007D8DECF0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002007D97EA20> root_client=<openai.OpenAI object at 0x000002007D807680> root_async_client=<openai.AsyncOpenAI object at 0x000002007D97CAA0> model_name='moonshotai/kimi-k2-instruct-0905' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [ ]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

# LCEL
### Prompt + LLM 모델 + OutputParser

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002007D8DECF0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002007D97EA20>, root_client=<openai.OpenAI object at 0x000002007D807680>, root_async_client=<openai.AsyncOpenAI object at 0x000002007D97CAA0>, model_name='moonshotai/kimi-k2-instruct-0905', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=

In [12]:
response = chain.invoke({"input":"LangChain은 무엇인가요?"})

In [13]:
print(type(response))
print(response)

<class 'str'>
LangChain은 “Large Language Model(LLM)을 애플리케이션에 쉽게 연결(chain)”해 주는 오픈소스 프레임워크입니다.  
핵심은 “LLM을 단순히 채팅용으로 쓰는 게 아니라, 외부 데이터·도구·API와 반복적으로 연결해 복잡한 업무 자동화를 만든다”는 것입니다.

1. 왜 나왔나?  
   - GPT 같은 LLM은 지식이 고정돼 있고, 실시간 데이터·사내 DB·사용자 행동에는 접근 못 함  
   - 복잡한 작업(“고객 질문 → DB 조회 → 보고서 생성 → 슬랙 전송”)은 한 번의 프롬프트로 안 됨  
   → 이 두 문제를 “Chain(체인)”으로 풀자

2. 핵심 개념 3줄 요약  
   1) Components: LLM, 프롬프트 템플릿, 출력 파서, 메모리, Document Loader 등 레고 블록  
   2) Chains: 블록들을 묶어 “Prompt → LLM → 후처리 → 다음 단계” 흐름을 반복  
   3) Agents: LLM이 “지금 어떤 도구(tool)를 써야 할까?”를 스스로 판단하며 루프 실행

3. 대표 모듈  
   - langchain-core: 범용 인터페이스  
   - langchain-community: 수백 개의 Loader/Tool 통합(Slack, Google, SQL, CSV…)  
   - langchain-openai / anthropic / google 등 벤더별 어댑터  
   - LangGraph: DAG·순환 워크플로 지원(2024 신규)  
   - LangServe / LangSmith: REST 배포·모니터링·디버깅 플랫폼

4. 실습 예 10초 컷  
```python
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI()
chain = (
    SystemMessage(conten